In [1]:
import torch, dgl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from dgl.data import DGLDataset
import dgl.function as fn
import re
import numpy as np
import pandas as pd
import os.path as osp
from colorama import Fore
from glob import glob
from random import shuffle
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

##### 1. graphSAGE

In [2]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    
    def __init__(self, in_feat:int, out_feat:int):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func = fn.copy_u('h', 'm'), 
                         reduce_func = fn.mean('m', 'h_N'))
                        #  reduce_func = fn.max('m', 'h_N'))
                        #  reduce_func = fn.sum('m', 'h_N'))
                        #  reduce_func = fn.sum('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)


class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

##### 2. Graph Dataset

In [3]:
class DkkdGraphDataset(DGLDataset):
    def __init__(self, root:str='/home/agent/Documents/graph/GNN/dataset/DKKD_graph'):
        super().__init__(name='dataset/DKKD_graph')
        self.root = root
        self.edges = glob(osp.join(root, '*.edges.csv'))
        self.nodes_feat = glob(osp.join(root, '*.nfeat.npy'))
        self.nodes_label = glob(osp.join(root, '*.idx.csv'))
    
    @staticmethod
    def _get_n_nodes(nodes_label:pd.DataFrame) -> int:
        r"""
        tính và kiểm tra số thứ tự của node
        """
        n_nodes = nodes_label['Id'].to_list()
        for i, idx in enumerate(n_nodes):
            assert i == idx, 'i != idx'
        return len(n_nodes)
    
    
    def __len__(self): return len(self.edges)
    
    def __getitem__(self, i):
        edgep = self.edges[i]
        nodes_feat = np.load(re.sub('.edges.csv$', '.nfeat.npy', edgep))
        nodes_label = pd.read_csv(
            re.sub('.edges.csv$', '.idx.csv', edgep), encoding='utf-8')
        n_nodes = self._get_n_nodes(nodes_label)
        
        nodes_label = nodes_label['label'].astype('category').cat.codes.to_list()
        edge = pd.read_csv(edgep, encoding='utf-8')
        
        g = dgl.graph((edge['src'], edge['dst']), num_nodes=n_nodes)
        g = dgl.to_bidirected(g)
        g = dgl.remove_self_loop(g)
        g.ndata['feat' ] = torch.from_numpy(nodes_feat )
        g.ndata['label'] = torch.tensor    (nodes_label)
        g.ndata['train_mask'] = torch.ones (n_nodes, dtype=torch.bool)
        g.ndata['val_mask'  ] = torch.zeros(n_nodes, dtype=torch.bool)
        g.ndata['test_mask' ] = torch.zeros(n_nodes, dtype=torch.bool)
        
        return g
    
    def process(self): ...
dataset = DkkdGraphDataset()

##### 3. Training function

In [8]:
def train(dataset:DGLDataset, model:nn.Module, epochs:int, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    all_logits = []
    best_acc = 0.9360
    
    for epoch in range(1, epochs+1):
        idxs = list(range(len(dataset)));shuffle(idxs)
        acc = 0
        for i in idxs:
            g = dataset[i]
            # g = g.to(device)
            features = g.ndata['feat'].float()
            labels = g.ndata['label']
            
            logits = model(g, features)
            pred = logits.argmax(1)
            loss = F.cross_entropy(logits, labels)
            acc += (pred == labels).float().mean()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            all_logits.append(logits.detach())
        
        acc = 1.*acc/len(dataset)
        if acc >= best_acc:
            best_acc = acc
            best_model = deepcopy(model)
            torch.save(model.state_dict(), 'weights/graphSAGE_best40.pth')
            
        if not epoch%32:
            print('Epoch {:<3d}: loss: {:.4f}, best {:.4f}, acc: {:.4f})'.format(
                            epoch, loss, best_acc, acc))

    torch.save(best_model.state_dict(), 'weights/graphSAGE_best40.pth')
    print('Last epoch {:<3d}, loss: {:.4f}, (best {:.4f}), acc: {:.4f})'.format(
                        epoch, loss, best_acc, acc))
    
    return best_model, best_acc

In [5]:
from dkkd_create_graph import create_cls_to_idx
classes, class_to_idx = create_cls_to_idx('dataset/DKKD/labels.txt')

num_classes = len(classes)  #48
print('num classes = ', num_classes)
model = GraphSAGE(772, 128, num_classes) # 

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


num classes =  48


In [11]:
best_model, best_acc = train(dataset, model, 100000, lr=0.0009)
torch.save(best_model.state_dict(), 'weights/graphSAGE_best40.pth')
print(best_acc)

Epoch 32 : loss: 0.1662, best 0.9308, acc: 0.9233)
Epoch 64 : loss: 0.3820, best 0.9336, acc: 0.9231)
Epoch 96 : loss: 0.1299, best 0.9336, acc: 0.9289)
Epoch 128: loss: 0.2548, best 0.9336, acc: 0.9301)
Epoch 160: loss: 0.1548, best 0.9336, acc: 0.9292)
Epoch 192: loss: 0.0058, best 0.9336, acc: 0.9280)


In [13]:
torch.save(best_model.state_dict(), 'weights/graphSAGE_best40.pth')

In [6]:
for i in range(dataset.__len__()):
    g = dataset[i]
    train(g, model, 100)

/home/agent/Documents/graph/GNN/dataset/DKKD_graph/0015.edges.csv


/home/agent/anaconda3/envs/graph/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch  25, loss: 0.0783, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  50, loss: 0.0474, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  75, loss: 0.0441, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch 100, loss: 0.0424, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
/home/agent/Documents/graph/GNN/dataset/DKKD_graph/0034.edges.csv


/home/agent/anaconda3/envs/graph/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch  25, loss: 0.1433, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  50, loss: 0.0560, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  75, loss: 0.0522, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch 100, loss: 0.0507, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
/home/agent/Documents/graph/GNN/dataset/DKKD_graph/0048.edges.csv


...

In epoch  25, loss: 7.5329, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  50, loss: 7.2380, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  75, loss: 6.9628, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch 100, loss: 6.7022, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
/home/agent/Documents/graph/GNN/dataset/DKKD_graph/0039.edges.csv
In epoch  25, loss: 2.9773, val acc: nan (best 0.0000), test acc: nan (best 0.0000)


/home/agent/anaconda3/envs/graph/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch  50, loss: 2.9143, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  75, loss: 2.8838, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch 100, loss: 2.8683, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
/home/agent/Documents/graph/GNN/dataset/DKKD_graph/0038.edges.csv


/home/agent/anaconda3/envs/graph/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch  25, loss: 3.5870, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  50, loss: 3.3392, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch  75, loss: 3.1268, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
In epoch 100, loss: 2.9470, val acc: nan (best 0.0000), test acc: nan (best 0.0000)
